# Importing pakages

In [1]:
import wandb
import pickle
import time
from src.datahandle import *
import time
import tensorflow as tf

# Setting up hyperparams

In [2]:
#hyper
directory = '../../extras/data/A_training_given/training_data/'
training_folder = 'training_data'
training_labels_file= 'training_norm.csv'
#
collected_directory = '../../extras/data/Z_collected_raw/' 
#
quiz_directory = '../../extras/data/C_testing_given/test_data/'
quiz_training_folder = 'test_data'
#configurations
conf_tracking = 0
seed = 0
data_shape = 'original'
parameter_init_scale = 0.01
split= 0.8
batch_size = 32
n_epochs = 5
lr = 0.0001

# Setting up DataLoaders

In [3]:
#dataloading object
training_object= DataLoader(
        directory,
        training_folder,
        training_labels_file
        )
collected_object = DataLoader(
        collected_directory        
        )

# Loading data information (not loading images yet)

In [4]:
collected_train, collected_test = collected_object.LoadCollectedData_info(
        split=split,
        batch_size=batch_size #for tf this batch size remains as 1
        )
#train test split
train,test = training_object.LoadModelData_info(
        split = split, 
        batch_size =batch_size)#for tf this batch size remains as 1

In [5]:
collected_train[0][1],train[1]

(array(['../../extras/data/Z_collected_raw/1648218352630_100_35.png',
        '1648218352630', '0.714', '0.868'], dtype='<U58'),
 array([['../../extras/data/A_training_given/training_data/training_data/4954.png',
         '4954', '0.625', '0.0'],
        ['../../extras/data/A_training_given/training_data/training_data/5492.png',
         '5492', '0.5', '0.901'],
        ['../../extras/data/A_training_given/training_data/training_data/3785.png',
         '3785', '0.75', '0.921'],
        ['../../extras/data/A_training_given/training_data/training_data/6825.png',
         '6825', '0.6875', '0.988'],
        ['../../extras/data/A_training_given/training_data/training_data/8808.png',
         '8808', '0.6875', '0.97'],
        ['../../extras/data/A_training_given/training_data/training_data/406.png',
         '406', '0.8125', '0.999'],
        ['../../extras/data/A_training_given/training_data/training_data/1192.png',
         '1192', '0.625', '0.922'],
        ['../../extras/data/A_traini

# Stacking collected and kaggle given data

In [6]:
train_XY = np.vstack((train,collected_train))
test_XY= np.vstack((test[0],collected_test[0]))

# Checking DataLoading functionality 

In [7]:
for i in range(1):
    test_batch_X,test_batch_Y = training_object.Load_batch(train_XY[i], data_shape=data_shape)
image_shape = test_batch_X.shape[1:]
image_shape

(240, 320, 3)

# Setting up data augmentation

In [9]:
data_augmentation = keras.Sequential(
  [
        layers.experimental.preprocessing.RandomRotation(0.2),
        layers.experimental.preprocessing.RandomZoom(0.2),
        #layers.experimental.preprocessing.RandomContrast(0.2),
  ]
)

# Setting up preprocessing for MobilenetV2

In [11]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

# Transfer Learning model

## to do
- [x] Seperate TF_cnn from TF_transfer
- [x] Remove rescaling in datahandling step (Now done by the preprocess layer)
- [x] Correctly rescale collected images labels (speed and angle)
- [x] Soften output
- [ ] Train the transfer learning model
- [ ] Tune the transfer learning model

In [12]:
ImageNetBase = tf.keras.applications.MobileNetV2(input_shape=image_shape,include_top=False,weights='imagenet')
ImageNetBase.trainable = False

In [13]:
feature_batch = ImageNetBase(test_batch_X)
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
output = global_average_layer(feature_batch)
inference_layer = tf.keras.layers.Dense(2)
ouput2 = inference_layer(output)


In [14]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dense1 = tf.keras.layers.Dense(256, activation='relu')
dense2 = tf.keras.layers.Dense(128, activation='relu')
inference_layer = tf.keras.layers.Dense(2)

In [15]:
inputs = tf.keras.layers.Input(shape=image_shape)
x = data_augmentation(inputs)
x = preprocess_input(x)
x = ImageNetBase(x)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = dense1(x)
x = dense2(x)
outputs = inference_layer(x)
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

In [16]:
model.layers

# Weights and Biases configuration

In [18]:
#wandb tracking
if conf_tracking:
    config = {
     "model_type" : 'Transfer Learning with MobileNetV2',
     "model_shape" : str(model.layers),
     "learning_rate": lr,
     "data_split": split,
     "batch_size": batch_size,
     "epochs": n_epochs,
    }
    run = wandb.init(project="Autonomous-driving", entity="mo379",config=config)

# Training Loop (With wandb logging)

In [ ]:
model_path = 'pkls/tf_cnn_augmented.pkl'
#model = tf.keras.models.load_model(model_path, custom_objects=None, compile=True, options=None)
test_batches = test_XY[0:2930].reshape(10,-1,4)
for _ in range(n_epochs):
    for i in range(1):
        X,Y = training_object.Load_batch(train_XY[i], data_shape=data_shape)
        train_metrics = model.train_on_batch(X,Y, return_dict=True) 

        X,Y = training_object.Load_batch(test_batches[np.random.randint(0,9)], data_shape=data_shape)
        test_metrics = model.test_on_batch(
            X, Y, sample_weight=None, reset_metrics=True, return_dict=True
        )
        
        if conf_tracking==1:
            wandb.log({"test_loss": test_metrics['loss']})
            wandb.log({"batch_loss": train_metrics['loss']})
#tf.keras.models.save_model(
#    model,
#    model_path,
#    overwrite=False,
#    include_optimizer=True,
#    save_format=None,
#    signatures=None,
#    options=None,
#    save_traces=True
#)


# Loading the Quiz data

In [ ]:
quiz_object= DataLoader(
    quiz_directory,
    quiz_training_folder,
)
quiz_train = quiz_object.LoadQuizData_info()
X,image_order = quiz_object.Load_batch_quiz(quiz_train,data_shape=data_shape)

# Making predictions

In [ ]:
prds = model.predict(X)
final_prd = np.column_stack((image_order,prds))
final_ordered = final_prd[final_prd[:, 0].argsort()]
df = pd.DataFrame(final_ordered, columns = ['image_id','angle','speed'])
df = df.astype({'image_id': 'int32'})
df.to_csv('submission.csv', index=False,)